In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: 12
### Team Member Names: Bill Bai, Soumik Debnath, Justin Yu
### Team Strategy Chosen: Risky (RISKY OR SAFE)

Below, we define start and end dates to get ticker data in order to create our portfolio. We defined it from 2020-11-26 to 2021-11-26, or a 1 year interval.

The reason why we chose to base our portfolio off of 1 year of historical data is because we wanted a timeframe that wasn't too long that our short-term risky portfolio would capture "too much" irrelevant data - for example, we chose to use standard deviation as a factor in creating our portfolio and using too long of a timeframe might cause our code to choose a stock that used to be volatile, but is now "safe". We also didn't want to use too short of a time interval since our portfolio uses correlation to choose stocks that tend to trade in the same direction - using too short of a timeframe would capture stocks that may be correlated for a short interval by pure coincidence or by broader-market forces.

Thus, the perfect interval we found after backtesting our code many times on different timeframes and lists of tickers happened to be 1 year. 1 year perfectly captures the stocks with the most momentum to make our portfolio risky, but also ensures that the stocks in our portfolio are correlated for a reason, rather than by pure coincidence or market forces.

Later on in our code, we'll also explain why we used daily data in calculating the risk metrics for our portfolio, rather than weekly or monthly data.

In [2]:
# Start and end date to base our portfolio tickers off of
start_date = "2020-11-26"
end_date = "2021-11-26"

Next we'll load the given list of tickers to choose our portfolio from.

In [3]:
# Read in tickers file and save as DF
tickers_path = "./Tickers.csv"
tickers_df = pd.read_csv(tickers_path, header=None).rename(columns={0: 'Ticker'})

In [4]:
# Display tickers dataframe
tickers_df

,Ticker
0,MTTRW
1,YUMC
2,PXS
3,ISEM
4,ING
5,ZVV
6,CNYA
7,MGTX
8,PIXY
9,VCIT


Here, we create an info column in our tickers dataframe, where each cell contains the yf.Ticker(ticker).info dict for each ticker. We implemented this to increase efficiency in our code, since calling .info takes a few seconds for each call, and our list of tickers is very long. Thus, it is more efficient to save the info dict in memory rather than performing an API call each time we need a stock's info.

In [5]:
# Create an info cell for each ticker, saving time when filtering the tickers by volume and country.
info_column = {}

# Add info to dict
for idx, row in tickers_df.iterrows():
    ticker = row[0]  # Get ticker name
    ticker = yf.Ticker(ticker)
    info_column[idx] = ticker.info
    print('.', end='')  # So we can tell if the code is running

# Create column for the info]
tickers_df['Info'] = pd.Series(info_column)
tickers_df

...........

,Ticker,Info
0,MTTRW,"{'zip': '94089', 'sector': 'Technology', 'full..."
1,YUMC,"{'zip': '200030', 'sector': 'Consumer Cyclical..."
2,PXS,"{'zip': '15125', 'sector': 'Industrials', 'lon..."
3,ISEM,"{'exchange': 'NGM', 'shortName': 'Invesco RAFI..."
4,ING,"{'zip': '1102 CT', 'sector': 'Financial Servic..."
5,ZVV,"{'symbol': 'ZVV', 'quoteType': 'EQUITY', 'exch..."
6,CNYA,"{'exchange': 'BTS', 'shortName': 'iShares MSCI..."
7,MGTX,"{'zip': '10016', 'sector': 'Healthcare', 'full..."
8,PIXY,"{'zip': '33131', 'sector': 'Industrials', 'ful..."
9,VCIT,"{'exchange': 'NGM', 'shortName': 'Vanguard Int..."


In the next two cells, we filter out stocks that are either delisted, have an average daily volume of less than 10,000, or aren't US-listed. This is per the requirements of the assignment.

We also create a dict called hist_dict that stores the yf.Ticker(ticker).history() dataframe for each ticker. This is also for efficiency since calling .history() takes times for Yahoo to return the data, so it's faster to just store the data in memory than to call it each time we use the data - which we do a lot of since we make many calculations for our portfolio.

In [6]:
# Dict to store the ticker history for each ticker.
hist_dict = {}


def filter_us_market(df):
    """
    Consumes a dataframe of tickers and returns a list of booleans representing whether the tickers are US-listed or not.
    :param df: DataFrame containing tickers
    :return: List of booleans representing whether the tickers are US-listed.
    """
    # Initialize mask list of booleans.
    mask = []
    for idx, row in df.iterrows():
        # Check whether stock is US-listed or not, and append the boolean to mask
        info = row['Info']
        if "market" in row['Info']:
            is_us_market = info['market'] == 'us_market'
            mask.append(is_us_market)
        else:
            mask.append(False)
    # Return mask
    return mask


def filter_volume(df):
    """
    Consumes a dataframe of tickers and returns a list of bo
    :param df:
    :return:
    """
    # Start and end date to check volume
    start = "2021-07-02"
    end = "2021-10-23"
    # Mask to filter out stocks
    mask = []
    for idx, row in df.iterrows():
        # Gets ticker history
        ticker = yf.Ticker(row[0])
        ticker_hist = ticker.history(start=start_date, end=end_date)
        # Gets subset of data to check volume
        volume_hist = ticker_hist.loc[
            (ticker_hist.index >= pd.to_datetime(start)) & (ticker_hist.index <= pd.to_datetime(end))]
        # Checks if average volume is not less than 10,0000
        valid_volume = volume_hist['Volume'].mean() >= 10000
        mask.append(valid_volume)
        if valid_volume:
            hist_dict[row[0]] = ticker_hist  # Adds the ticker history dataframe to hist_dict
    return mask

In [7]:
# Filter out non-US stocks / delisted stocks
tickers_df = tickers_df.loc[filter_us_market(tickers_df)]
# Filter out stocks without valid volume
tickers_df = tickers_df.loc[filter_volume(tickers_df)]

- ZVV: No data found for this date range, symbol may be delisted


In [8]:
# Reset index on the filtered dataframe
tickers_df = tickers_df.reset_index(drop=True)
tickers_df

,Ticker,Info
0,MTTRW,"{'zip': '94089', 'sector': 'Technology', 'full..."
1,YUMC,"{'zip': '200030', 'sector': 'Consumer Cyclical..."
2,PXS,"{'zip': '15125', 'sector': 'Industrials', 'lon..."
3,ING,"{'zip': '1102 CT', 'sector': 'Financial Servic..."
4,CNYA,"{'exchange': 'BTS', 'shortName': 'iShares MSCI..."
5,MGTX,"{'zip': '10016', 'sector': 'Healthcare', 'full..."
6,PIXY,"{'zip': '33131', 'sector': 'Industrials', 'ful..."
7,VCIT,"{'exchange': 'NGM', 'shortName': 'Vanguard Int..."
8,SRE,"{'zip': '92101', 'sector': 'Utilities', 'fullT..."


Next, we reformatted the data such that it contains ticker history closing prices for the 1 year interval. For this data, we kept it as daily data rather than resampling it to monthly or weekly data.

The reason why we chose to use daily data to calculate the metrics for our portfolio (such as STD and Correlation) is because we found that it produced the best results when backtesting our short-term risky portfolio. We decided against using monthly closing prices for our portfolio since using 1 year of historical data would only give us 12 closing prices to decide our portfolio off of - which is too little. We also tried using weekly data and decided against it since we found that it didn't capture the most short-term volatile stocks. Weekly data tended to give us stocks that were volatile by coincidence (such as news causing a sharp drop or rise in stock price), rather stocks that are intrinsically volatile (like oil stocks, which are volatile not by coincidence but because oil is a volatile asset). Thus, we found that daily data gave us the most intrinsically short-term volatile stocks to build our risky portfolio.

In [9]:
def create_df(df):
    """
    This function iterates through the filtered list of stocks and creates a new dict with weekly ticker closing prices
    :param df: filtered stocks DataFrame
    :return: dict containing ticker closing prices resampled to weekly data
    """
    dic = {}
    for i in range(len(df)):
        tick = df.iloc[i, 0]
        # get ticker history data for the ticker
        t_hist = hist_dict[tick]
        # resample data to
        dic[tick] = t_hist.Close
    return dic


#creates the dataframe price using the function create_df
prices = pd.DataFrame(create_df(tickers_df))
prices

,MTTRW,YUMC,PXS,ING,CNYA,MGTX,PIXY,VCIT,SRE
Date,,,,,,,,,
2020-11-27,NaN,57.928795,0.882,9.507059,40.076794,13.900000,2.510,94.685852,126.188179
2020-11-30,NaN,55.925091,0.940,9.155999,39.582020,14.120000,2.380,94.851776,123.305756
2020-12-01,NaN,57.700649,0.892,9.469106,40.710106,13.700000,2.360,94.586754,123.576591
2020-12-02,NaN,56.163151,0.920,9.592451,40.729897,14.110000,2.350,94.567184,127.126404
2020-12-03,NaN,56.351620,0.934,9.488082,40.680416,13.980000,2.320,94.772568,127.687408
...,...,...,...,...,...,...,...,...,...
2021-11-18,15.550000,53.339752,0.682,15.060000,42.830002,19.760000,0.992,93.370003,122.739998
2021-11-19,16.070000,53.369686,0.679,14.450000,43.299999,18.650000,0.971,93.519997,122.760002
2021-11-22,14.507000,53.230000,0.673,14.640000,43.650002,18.600000,0.890,92.900002,124.949997


In the next 2 cells, we obtain the stock with the highest risk (calculated using standard deviation). This will be a key part in forming our portfolio, as explained in the next annotation cell.

In [10]:
def generate_std(df):
    """
    Creates a dict of tickers and their standard deviations for the given closing prices
    :param df: Filtered DataFrame of tickers
    :return: Dict of tickers and their STDs
    """
    column = df.columns
    stdlst_dic = {}
    # Calculate the STD for each ticker
    for i in range(len(column)):
        temp = df[str(column[i])].pct_change().std()
        stdlst_dic[str(column[i])] = temp
    return stdlst_dic


#creates the dataframe slst which is a dictionary containing the tickers and their std
slst = generate_std(prices)
slst

{'MTTRW': 0.0892254509771861,
 'YUMC': 0.014315493745677081,
 'PXS': 0.0847520699935749,
 'ING': 0.019183774190791664,
 'CNYA': 0.013830901935157102,
 'MGTX': 0.037985766868854735,
 'PIXY': 0.06858140581324966,
 'VCIT': 0.002566925435147436,
 'SRE': 0.012676385404511076}

In [11]:
#function to find the highest std
def get_highest_std(dic):
    """
    Consumes a dictionary of tickers and their STDs and returns the ticker with the highest STD in the dict
    :param dic: Dict of tickers and their STDs
    :return: Ticker with the highest STD
    """
    std = 0
    tick = ""
    # Calculates the highest STD
    for i in dic:
        if std < dic[i]:
            std = dic[i]
            tick = i
    return tick


#checking what the highest std is
get_highest_std(slst)

'MTTRW'

Next we'll generate a correlation matrix for all the stocks

In [12]:
#creating a correlation dataframe
corr_df = prices.corr()
corr_df

,MTTRW,YUMC,PXS,ING,CNYA,MGTX,PIXY,VCIT,SRE
MTTRW,1.000000,-0.573786,-0.185994,0.613578,0.078642,0.549103,-0.613099,0.157793,-0.335099
YUMC,-0.573786,1.000000,0.064038,0.158054,0.285528,-0.208581,0.263343,-0.060410,0.541816
PXS,-0.185994,0.064038,1.000000,-0.398068,0.370015,0.126407,0.642069,-0.332790,-0.436046
ING,0.613578,0.158054,-0.398068,1.000000,-0.107830,0.219927,-0.749543,-0.198693,0.493866
CNYA,0.078642,0.285528,0.370015,-0.107830,1.000000,0.242679,0.347012,0.233942,-0.257934
MGTX,0.549103,-0.208581,0.126407,0.219927,0.242679,1.000000,-0.069285,-0.145946,-0.253653
PIXY,-0.613099,0.263343,0.642069,-0.749543,0.347012,-0.069285,1.000000,-0.202839,-0.261999
VCIT,0.157793,-0.060410,-0.332790,-0.198693,0.233942,-0.145946,-0.202839,1.000000,-0.100221
SRE,-0.335099,0.541816,-0.436046,0.493866,-0.257934,-0.253653,-0.261999,-0.100221,1.000000


In the next cells, we use correlation to calculate stocks that trade in the same direction as the stock with the highest STD, in order to create a portfolio where all the stocks are strongly positively correlated with each other. In our original code, we instead compiled a list of stocks in the same industry to achieve the opposite affect of inter-industry diversification. However, we then realized that it would make more sense to choose a basket stocks that are strongly positively correlated with each other, since it would acheive the same effect as an intra-industry portfolio, but would give us a larger variety of risky stocks to choose from so we can choose the riskiest of stocks.

In [13]:
def lister(corr, highest_std):
    """
    Creates a dict of stocks that are either positively or negatively correlated with the highest std stock
    :param corr: Correlation matrix of all stocks
    :param highest_std: Ticker of stock with the highest
    :return:
    """
    list1 = []
    list2 = []
    columns = corr.columns
    for i in range(len(corr.index)):
        c = corr[highest_std].iloc[i]
        tick = columns[i]
        if c > 0:
            list1.append(tick)
        elif c < 0:
            list2.append(tick)
    dic = {"positive": list1, "negative": list2}
    return dic


#creates dictionary cor_list which calls the function lister
# contains two lists which are positive correlating and negative correlating with the inputed stock
cor_list = lister(corr_df, get_highest_std(slst))
cor_list

{'positive': ['MTTRW', 'ING', 'CNYA', 'MGTX', 'VCIT'],
 'negative': ['YUMC', 'PXS', 'PIXY', 'SRE']}

We then sort the both baskets of positively and negatively correlated stocks by their STDs in descending order so we can choose stocks that have the highest STD and are also strongly positively correlated with our keystone stock - the stock that has the highest std and weighting in our final portfolio.

In [14]:
def quicksort(tick_lst, std_list):
    """
    Implementation of quicksort to sort a list of tickers based on their std
    :param tick_lst: Ticker list
    :param std_list: List of respective STDs of the tickers.
    :return: Sorted list of tickers
    """
    sequence = tick_lst
    length = len(sequence)
    if length <= 1:
        return sequence
    else:
        pivot = sequence.pop()

    items_greater = []
    items_lower = []

    for item in sequence:
        i = std_list[item]
        pivoter = std_list[pivot]
        if i < pivoter:
            items_lower.append(item)
        else:
            items_greater.append(item)

    return quicksort(items_greater, slst) + [pivot] + quicksort(items_lower, slst)


positive = quicksort(cor_list["positive"], slst)
negative = (quicksort(cor_list["negative"], slst))

positive

['MTTRW', 'MGTX', 'ING', 'CNYA', 'VCIT']

We also have a function to ensure that our code doesn't acheive the opposite of a risky portfolio - in the case that there is less than 10 stocks that are positively correlated with the stock with the highest STD, it might choose stocks that are negatively correlated and create a balanced portfolio. To overcome this, we make sure that the STD of the basket of stocks WITH the highest correlation is higher than the STD of the basket of stocks that are strongly negatively correlated against the stock with the highest STD. In the case that this is false, then our portfolio becomes the stocks with the strongly negatively correlated stocks. In either case, this ensures that our portfolio will have the highest STD possible, and all the stocks in the portfolio trade in the same direction.

In [15]:
def meanstd(lst, count, std_list, sum, lst_tick):
    """
    Recursively calculates the average STD of the top 10 stocks in each list of positive and negative correlated stocks.
    :param lst: List of tickers
    :param count: Iterative counter
    :param std_list: Ticker's stds
    :param sum: Sum of the STD
    :param lst_tick: Second list of tickers
    :return: Average STD of stocks in a list
    """
    if 0 == len(lst):
        return {"list": lst_tick, "mean": (sum / count)}
    elif 10 == count:
        return {"list": lst_tick, "mean": (sum / 10)}
    else:
        sum = sum + std_list[lst[0]]
        lst_tick.append(lst[0])
        return meanstd(lst[1:], (count + 1), std_list, sum, lst_tick)


pm = meanstd(positive, 0, slst, 0, [])
nm = meanstd(negative, 0, slst, 0, [])

len(nm["list"])

4

In [16]:
def hstd(p, n):
    """
    Gets the higher average std of the positively and negatively correlated stocks
    :param p: Positively correlated stock std
    :param n: Negatively correlated stock std
    :return: Higher average STD
    """
    pm=p["mean"]
    nm=n["mean"]
    pl=len(p["list"])
    nl=len(n["list"])
    p["oppo"]=negative
    n["oppo"]=positive
    
    if pm > nm and pl >=7:
        return p
    elif nm > pm and nl >= 7:
        return n
    elif pl == nl:
        if pm > nm:
            return p
        elif nm > pm:
            return n
    elif pl > nl:
        return p
    elif nl > pl:
        return n
    else: 
        return ("bruh")
        


port_list = hstd(pm, nm)
port_list

{'list': ['MTTRW', 'MGTX', 'ING', 'CNYA', 'VCIT'],
 'mean': 0.03255856388142741,
 'oppo': ['PXS', 'PIXY', 'YUMC', 'SRE']}

In the case that the list of tickers given to us is less than 20 (such as 6 positively correlated stocks and 5 negatively correlated stocks), our portfolio appends the negatively correlated stocks with the lowest STD to our positively correlated portfolio. This ensures that our portfolio has at least 10 stocks and the negatively correlated stocks do not affect our portfolio performance as much.

In [17]:
def portlength(port):
    """
    Appends weakly negatively correlated stocks to our portfolio if there aren't enough positively correlated stocks in our portfolio.
    :param port: Portfolio tickers
    :return: New portfolio
    """
    lster=port["list"]
    
    if len(lster) == 10:
        return lster
    else:
        l=10-len(lster)
        for i in range(0,l) :
            lster.append(port["oppo"][-1 *(i+1)])
        return lster



finalport_lst=portlength(port_list)
finalport_lst

IndexError: list index out of range

In [ ]:
lstofstocks = finalport_lst
lstofstocks

Finally, we set our weights in this cell and backtest our portfolio over 2021-11-01 to 2021-11-26. For our weightings, we made them

35% 25% 5% 5% 5% 5% 5% 5% 5% 5%

Where the stock with the highest STD in our portfolio is weighted at 35% and the stock with the lowest STD is weighted at 5%. This ensures that overall, our portfolio has the highest STD as allowed by the requirements of the assignment (limited by the weightings), by giving the highest weighting to the riskiest stock. That way, a volatile move by the stock with the highest std will affect our portfolio the most.

In [ ]:
# Using the 35% distribution

startdate = '2021-11-01'
enddate = '2021-11-26'

#Determine number of shares bought for each stocks under 35% weighted profolio ($100,000)
# Assuming that the stocks i the list are given in increasing order
NumOfShares = []  # number of shares will be printed in the list according to the corrosponding order


# Loopsahre function takes a list the starting date and the ending date, and produces the number of shares for each stocks in the list given
# Since the stocks given in the list is already sorted in decreasing order of standard deviation, we con proceed to apply the
# 35% percent distribution method (explained in the report)
def loopshare(lst, date_start, date_end):
    for i in range(len(lst)):
        if (i < 1):
            NumOfShares.append(35000 / ((yf.Ticker(lst[i]).history(start=date_start, end=date_end)).iloc[0, 3]))
        elif (i < 2):
            NumOfShares.append(25000 / ((yf.Ticker(lst[i]).history(start=date_start, end=date_end)).iloc[0, 3]))
        else:
            NumOfShares.append(5000 / ((yf.Ticker(lst[i]).history(start=date_start, end=date_end)).iloc[0, 3]))
    return NumOfShares


loopshare(lstofstocks, startdate, enddate)

listOfClose = []  #list of dataframe


#Extracting the closed data

# Funcation loopClose takes a lst of tickers, start date, and a end date, and produces a list of closing price of each stocks in the list date
# Withtin the start and end date
def loopClose(lst, start_date, end_date):
    for i in range(len(lst)):
        listOfClose.append(pd.DataFrame((yf.Ticker(lst[i]).history(start=start_date, end=end_date))['Close']))


loopClose(lstofstocks, startdate, enddate)

#Setting up a list before adding it to the dataframe
listBefore = []


#Function loopCloseBefore, creates a list of closing prices of tickers that is ready to be put into a dataframe
def loopCloseBefore(lst):
    for i in range(len(lst)):
        if (i == 0):
            listBefore.append(lst[i])
        else:
            listBefore.append(lst[i]['Close'])


loopCloseBefore(listOfClose)

#Create the dataframe for tracking the portfolio

#concating all the list of closign prices of stocks
finalPortfolio = pd.concat(listBefore, join='inner', axis=1)

#Renaming the column names to the corrosponding stocks
finalPortfolio.columns = lstofstocks

#Adding portfolio

#Adding the column portfolio
finalPortfolio['portfolio'] = finalPortfolio[str(finalPortfolio.columns[0])] * NumOfShares[0] + finalPortfolio[
    str(finalPortfolio.columns[1])] * NumOfShares[1] + finalPortfolio[str(finalPortfolio.columns[2])] * NumOfShares[2] + \
                              finalPortfolio[str(finalPortfolio.columns[3])] * NumOfShares[3] + finalPortfolio[
                                  str(finalPortfolio.columns[4])] * NumOfShares[4] + finalPortfolio[
                                  str(finalPortfolio.columns[5])] * NumOfShares[5] + finalPortfolio[
                                  str(finalPortfolio.columns[6])] * NumOfShares[6] + finalPortfolio[
                                  str(finalPortfolio.columns[7])] * NumOfShares[7] + finalPortfolio[
                                  str(finalPortfolio.columns[8])] * NumOfShares[8] + finalPortfolio[
                                  str(finalPortfolio.columns[9])] * NumOfShares[9]

As seen below, over the course of 1 month from 2021-11-01 to 2021-11-26, our portfolio fell 5%. At the lowest, it fell nearly 8%. The backtesting of our portfolio on historical data reveals that it is very risky as desired.

In [ ]:
finalPortfolio

In [ ]:
# Portfolio starting date
comp_date = '2021-11-26'

Please note that while we were testing our code, Yahoo Finance sometimes updates their data unpredictably, so we couldn't always get the closing prices for the current day. However, this shouldn't be a problem when you run the code past the 26th.

In [ ]:
def price_data(lst,date):
    dic={}
    for i in range(len(lst)):
        t=yf.Ticker(lst[i])
        t_hist=t.history(start=date).iloc[0]
        dic[lst[i]]=t_hist.Close
    return dic

comp_closing=pd.DataFrame(price_data(lstofstocks, comp_date), index=[pd.to_datetime(comp_date)])

def calc_shares(df):
    dic={}
    for i in range(len(df.index)):
        for j in range(len(df.columns)):
            if j <= 0:
                dic[df.columns[j]]=35000/df.iloc[i,j]
            elif j <= 1:
                dic[df.columns[j]]=25000/df.iloc[i,j]
            else:
                dic[df.columns[j]]=5000/df.iloc[i,j]
    return dic

shares=calc_shares(comp_closing)
sa=pd.DataFrame.from_dict(shares,orient="index")

comp_closing=comp_closing.transpose()
FinalPortfolio=comp_closing

FinalPortfolio["Shares"]=sa[0]
FinalPortfolio=comp_closing.reset_index()
FinalPortfolio.columns=["Ticker","Prices","Shares"]
FinalPortfolio["Values"]=FinalPortfolio["Prices"]*FinalPortfolio["Shares"]
FinalPortfolio["Weight"]=(FinalPortfolio["Values"]/100000)*100
FinalPortfolio

In [ ]:
Stocks=pd.DataFrame(FinalPortfolio["Ticker"])
Stocks["Shares"]=FinalPortfolio["Shares"]
Stocks

In [ ]:
Stocks.to_csv('Stocks_Group_12.csv', index=False)
pd.read_csv('Stocks_Group_12.csv')

## How we came up with our list of tickers
When thinking about what a risky portfolio is, we directed our focus to creating the least diverse portfolio made with the most volatile stocks. Instantly, we wanted to create a portfolio with 10 volatile stocks with high stds over a reasonable time interval.

Our original thought process was to compare industries and find the industry with stocks which have the highest std. This would create very little diversification as stocks in the same industry tend to trend the same way. However, this method was ineffective for numerous reasons. The code would be far less efficient as we must call the info dictionary to get the industry and track down how many industries there are, moreover, we must check if there are even 10 stocks in each industry. Furthermore, one overtly volatile stock in one industry would lead us to base the portfolio on that industry; this is a problem as the industry might be relatively safe other than that one volatile stock creating a safe portfolio consequently.

Hence, we shifted our approach to look at correlation of each stock to the stock with the highest std. This would grant us to create two groups of stocks; stocks that positively correlate with the stock with the highest std and stocks that negatively correlate. The two lists of stocks were then reorganized by highest to lowest std and the top 10 were picked from each list. This method grants us to create two lists of stocks which are very risky and have a history of correlating with each other (less diversification within each list). We then take the list of stocks which have the higher average std. If the group of stocks with the higher average std has less than 10 stocks, we just take stocks from the bottom of the list of the original organized list of oppositely correlating stocks. In other words, from the list of tickers, we have limited it down to 10 stocks which all relatively correlate in the same direction and are very volatile. Correlation is better than industry as industries also correlate with each other so by being inclusive of multiple industries through correlation, we get access to other volatile stocks in other industries.


## 35% diversification method:
In the final part of the program, we’ve implemented a diversification method to determine the proportion of stocks bought using the 100,000, which we called the 35% diversification method. Our goal for the group is to create the most risky portfolio possible. As stated above, our condition for a risky portfolio is to have the least amount of variation of stocks possible, a low volume for each stock, and a high standard deviation for each stock, more importantly, the stocks in the portfolio have to correlate in the same direction.
However, in the rules for the competition, we are required to have a minimum of 10 stocks, and no value of a specific stock in the portfolio should weigh over 35%. Therefore, according to our condition for a high risk portfolio, we decided to make a portfolio with only 10 stocks. Another rule for the assignment is that each stock must make up a minimum of (100/(2n))% of the portfolio therefore, this leaves us with a portfolio of 10 stocks, and no stocks could be weighted higher than 35% or lower than 5%.
At the point of the code before implementing the 35% diversification method, we’ve already got a list of 10 stocks that are given in a decreasing order of standard deviation(calculated by previous historic data), and also correlated in the same direction, that means the first stock in the list is the most volatile, where as the last stock is the most volatile, but among the ten stocks. Since we know the list is given in a decreasing order, we wish to allocate our proportion of investment in a decreasing order as well, meaning that we put the most money in the first stock in the list and the least in the last stock, which optimizes the risk. Keeping in mind the fact that no values of stocks can make up more than 35%, and no values of stocks and be below 5%, by mathematical calculation, this leaves us with the optimum weighting of 35% for the first stock, 25% for the second stocks, and 5% each for the rest of the 8 stocks. In this case we can guarantee that 60% of our portfolio is driven by the two riskiest stocks, that correlates the same way, and the rest 40% of our portfolio is driven by 8 stocks that are less risky, but also correlates the same way. In this case, we created a portfolio that follows our condition for a risky portfolio, which is more concentrated, and have the least amount of variation stocks possible, but also not violating the rules for the assignment at the same time.

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Bill:
Filtration code, annotation in the actual code and overall error checking.

Soumik:
Correlation code and generating the actual list of tickers to be used. Also worked on the final written portion.

Justin:
Code for calculating the weighting and outputting it to the csv. ALso worked on the final written portion.